# Dialogue Summarization with transformer-based pre-trained models

This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with the Splunk Machine Learning Toolkit (MLTK) Container for TensorFlow. This script contains an example of how to conduct **End-to-End automatic summarization task over English dialogue transcripts** using the transformers library.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
# import sys
# sys.path.insert(1, '/opt/conda/lib/python3.8/site-packages')

import json
import numpy as np
import pandas as pd
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import os
path = os.path.abspath(np.__file__)
print("numpy path in runtime: " + path)
print("numpy version in runtime: " + np.__version__)

# tokenizer = AutoTokenizer.from_pretrained("/opt/conda/lib/python3.8/site-packages/samsum_tok", local_files_only=True)
# summarizer = AutoModelForSeq2SeqLM.from_pretrained("/opt/conda/lib/python3.8/site-packages/samsum", local_files_only=True)

# tokenizer = AutoTokenizer.from_pretrained("philschmid/bart-large-cnn-samsum")
# # tokenizer.save_pretrained("/opt/conda/lib/python3.8/site-packages/samsum_tok")
# model = AutoModelForSeq2SeqLM.from_pretrained("philschmid/bart-large-cnn-samsum")
# model.save_pretrained("/opt/conda/lib/python3.8/site-packages/samsum")

MODEL_DIRECTORY = "/srv/app/model/data/"

numpy path in runtime: /opt/conda/lib/python3.8/site-packages/numpy/__init__.py
numpy version in runtime: 1.22.1


In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.22.1
pandas version: 1.4.3


In [3]:
# from transformers import pipeline
# summarizer = pipeline("summarization", model="lidiya/bart-base-samsum")

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| makeresults
| eval text = "Client: Hello Customer Service Representative: Welcome, Mr. Client, how are you today? Client: Welcome, I’m fine. Customer Service Representative: With you from the complaints and suggestions department, how can I help you today, Mr. Client? Client: I have a big problem with the product I bought from your site this week Customer Service Representative: Moments with me Mr. Client, I will review your request now Client: Ok Customer Service Representative: Your request number 100145 was on Wednesday correct Mr. Client?  Client: Yup Customer Service Representative: Ok, Mr. (client’s name) can you explain to me what is the problem exactly?  Client: I didn’t like the product and I didn’t get it as the same picture on the company website, you are liars! Customer Service Representative: I apologize very much to you Mr. (client’s name) but on our website, we put pictures of real products only, and the proof is the positive comments and evaluations from all our customers in Saudi Arabia Client: I don’t believe it; I believe my eyes who see the product Customer Service Representative: Sorry, Mr. (client’s name) I’m not accusing you of lying at all, and I will solve this problem immediately, can you please explain to me in detail how the product is different from the picture? Client: In the picture, the size of the product is bigger than what I received Customer Service Representative: Ok, Mr. Client if you opened our website, you would find the sizes written in detail in the product description, if you can look at it now while you are with me on the line? Client: Ok Customer Service Representative: Take your time, Mr. (client’s name) Client: it is Written but the product in the picture is very big than the size that is written in the description? Customer Service Representative: This is because of the angle of photography Mr. (client’s name) because we in the company must show all the details of the product to the customer, so the pictures are a little close to the product to show all the details to the customers, and we provide the product with a specific and clear description of materials and sizes and advantages and everything so that the customer knows the product that he will buy, are you still with me Mr. (client’s name)? Client: yes Customer Service Representative: Are there any other differences than the size Mr. (client’s name)?  Client: I don’t know how to use this product; I mean the way I used it and it didn’t work Customer Service Representative: There is no problem Mr. (client’s name) I will send you an accurate description of the method of use and will follow up with you one of the customer service representatives in the department of inquiries to clarify all the details and information about the product  Client: Pretty good! Customer Service Representative: Do you have any other complaints, Mr. Client? Client: I wanted to complain to the shipping representative who got me this product Customer Service Representative: Moments with me, Mr. Client now I will determine who is responsible for your shipping operation"
| fit MLTKContainer algo=transformers_dialogsum_en mode=stage epochs=100 text into app:transformers_dialogsum_model as text

In [4]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [5]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("transformers_dialogsum_model")
print(df)
print(df.shape)
print(str(param))

                                                text
0  Client: Hello Customer Service Representative:...
(1, 1)
{'options': {'params': {'algo': 'transformers_dialogsum2', 'mode': 'stage', 'epochs': '100'}, 'args': ['text'], 'feature_variables': ['text'], 'model_name': 'transformers_dialogsum_model', 'output_name': 'text', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '600', 'max_inputs': '100000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '30', 'max_score_time': '600', 'use_sampling': 'true'}, 'kfold_cv': None}, 'feature_variables': ['text']}


## Stage 2 - create and initialize a model

In [6]:
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    # Load English parser and text blob (for sentiment analysis)
    print("debug: start model initialization")
    model = {}
    model["tokenizer"] = AutoTokenizer.from_pretrained("philschmid/bart-large-cnn-samsum")
    model["summarizer"] = AutoModelForSeq2SeqLM.from_pretrained("philschmid/bart-large-cnn-samsum")
    # model["tokenizer"] = AutoTokenizer.from_pretrained("/srv/app/model/data/samsum_tok")
    # model["summarizer"] = AutoModelForSeq2SeqLM.from_pretrained("/srv/app/model/data/samsum")
    print("debug: model initialized")
    return model

In [7]:
model = init(df,param)

debug: start model initialization
debug: model initialized


## Stage 3 - fit the model

Note that for this algorithm the model is pre-trained and therefore this stage is a placeholder only

In [8]:
# returns a fit info json object
def fit(model,df,param):
    returns = {}
    return returns

## Stage 4 - apply the model

In [9]:
def apply(model,df,param):
    
    print("debug: start apply")
    
    X = df[param['feature_variables']].values.tolist()
    temp_data=list()
    
#     tokenizer = AutoTokenizer.from_pretrained("/srv/app/model/data/samsum_tok")
#     summarizer = AutoModelForSeq2SeqLM.from_pretrained("/srv/app/model/data/samsum")
    print("debug: read input")
    print("numpy version: " + np.__version__)
    for i in range(len(X)):
        print("debug: start tokenizing")
        input_ids = model["tokenizer"](
            str(X[i]), return_tensors="pt"
        ).input_ids  # Batch size 1
        print("debug: finish tok and start summarizing")
        outputs = model["summarizer"].generate(input_ids, max_length=100)
        print("debug: finish summarizing and start decoding")
        summary = model["tokenizer"].decode(outputs[0], skip_special_tokens=True)
        print("debug: finish decoding")
        temp_data.append(summary)
        
    column_names=["summaries"]
    returns=pd.DataFrame(temp_data, columns=column_names)
    print("debug: finished")
        
    return returns

In [10]:
returns = apply(model,df,param)
print(returns["summaries"][0])

debug: start apply
debug: read input
numpy version: 1.22.1
debug: start tokenizing
debug: finish tok and start summarizing
debug: finish summarizing and start decoding
debug: finish decoding
debug: finished
Client has a problem with the product he bought from the company this week. The size of the product in the picture is bigger than the size that is written in the description. Customer Service Representative will send him an accurate description of the method of use and he will follow up with one of the customer service representatives in the department of inquiries to clarify all the details.


## Stage 5 - save the model

In [11]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # model will not be saved or reloaded as it is pre-built
    return model

## Stage 6 - load the model

In [11]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    # model will not be saved or reloaded as it is pre-built
    return model

## Stage 7 - provide a summary of the model

In [12]:
# return model summary
def summary(model=None):
    returns = {}
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code